In [2]:
import requests
import time 
import json
import hashlib
from pprint import pprint
import urllib.parse
import binascii
from requests.packages.urllib3 import disable_warnings
from connection import ConnectionTask
c=ConnectionTask()
disable_warnings()

ELE_HOST = 'https://openapi.ele.me/v2'
ELE_CONSUMER_KEY = '5634882370'
ELE_CONSUMER_SECRET = 'f754a8c2efe5a98fba814cc94971dcd6793945c0'

def concat_params(params):
    pairs = []
    for key in sorted(params):
        if key == 'sig':
            continue
        val = params[key]
        if isinstance(val,str):
            val = urllib.parse.quote_plus(val)
        pairs.append("{}={}".format(key, val))
    return '&'.join(pairs)
def gen_sig(path_url, params, consumer_secret):
    params = concat_params(params)

    to_hash = hashlib.sha1(binascii.hexlify('{}?{}{}'.format(
        path_url, params, consumer_secret
    ).encode('utf-8')))
    sig = to_hash.hexdigest()
    return sig

In [3]:
# 订单状态
def order_status_fetch(data):
    params = {
        "timestamp": int(time.time()),
        "consumer_key": ELE_CONSUMER_KEY
    }
    url = ELE_HOST + "/order/{}/status/".format(data['partnerId'])
    params["sig"] = gen_sig(url, params, ELE_CONSUMER_SECRET)
    resp = requests.get(url, params=params, verify=False).json()
    data['statusCode']=resp['data']['status_code']
    return data
# print(resp)
# print(resp['data']['status_code'])

In [9]:
order_status_fetch({"orderId":"ala203675812173684","partnerId":"3019967166476323978"})

{'orderId': 'ala203675812173684',
 'partnerId': '3019967166476323978',
 'statusCode': -1}

In [ ]:
start,end,count = time.mktime((2018,1,8,0,0,0,0,0,0))*1000,time.mktime((2018,1,22,0,0,0,0,0,0))*1000,0
order_list=[]
for item in c.pmdb.order_waimai.find({'status':4,'createdAt':{'$gte':start,'$lt':end}},{'id':1,'partnerId':1,'_id':0,'completedAt':1}):
    resp=order_status_fetch(item)
    count+=1
    if resp['statusCode'] != 2:
        print(resp)
        order_list.append(resp)
    print('completedAt:{},count:{}'.format(time.strftime("%Y-%m-%d",time.localtime(item.get('completedAt',0)/1000)),count),end='\r')

In [ ]:
[item['id'] for item in order_list]

In [8]:
# 检查数据库是否有已退款，但饿了么为已完成订单。
start,end = time.mktime((2018,2,27,0,0,0,0,0,0))*1000, time.mktime((2018,2,28,0,0,0,0,0,0))*1000
count=0
for item in c.pmdb.order_waimai.find({'status':7,'createdAt':{'$gte':start}},{'id':1,'partnerId':1,'_id':0,'timeline':1}):
    resp = order_status_fetch(item)
    if resp['statusCode']==2:
        print(item['id'],resp,item['timeline'])
    count+=1
    print(count,end='\r')
    

In [ ]:
# 补偿订单
start=time.mktime((2017,9,28,0,0,0,0,0,0))*1000
end=time.mktime((2017,9,30,6,0,0,0,0,0))*1000
# print(start,end)
waimai_order_message=[{'orderId':item['id'],'partnerId':item['partnerId'],'displayStatus':item['displayStatus']} for item in c.pmdb.order_waimai.find({'createdAt':{'$gte':start,'$lt':end}, 'status':{'$in':[9]}, 'displayStatus':"配送中"},{'_id':0,'id':1,'partnerId':1,'displayStatus':1})]
for item in waimai_order_message:
    item=order_status_fetch(item)

In [ ]:
order = [item['orderId'] for item in waimai_order_message if item['statusCode'] ==2]

In [ ]:
for item in c.pmdb.order_waimai.find({'id':{'$in':order}},{'_id':0,'id':1,'appCode':1}):
    refund_id=c.prdb_paym(0).get('r_mapper:{}:{}'.format(item['appCode'],item['id']))
    if not refund_id:
        continue
    c.prdb_paym(0).delete('r_mapper:{}:{}'.format(item['appCode'],item['id']))
    c.prdb_paym(2).delete('r:{}:{}'.format(item['appCode'],refund_id))
    print(item['id'],item['appCode'],refund_id,)


In [ ]:
# ala202037982041713 91ala ala222037984007795

In [ ]:
# 获取城市列表city-list
# 通过接口获取饿了么支持的城市列表

params = {
    "timestamp": int(time.time()),
    "consumer_key": ELE_CONSUMER_KEY
}
url = ELE_HOST + "/cities/"
params["sig"] = gen_sig(url, params, ELE_CONSUMER_SECRET)
resp = requests.get(url, params=params).json()
# {'code': 200, 'data': {'cities': [{'abbr': 'SH', 'id': 1, 'name': '上海'},...]}}
print(resp['data']['cities'][0])

In [ ]:
ele_citys={item['name']:item['id'] for item in resp['data']['cities']}
with open('/data/ele_citys.json','w') as f:
    json.dump(ele_citys,f)

In [ ]:
# 搜索餐厅restaurants-search
# 根据指定的条件查询餐厅，返回餐厅信息列表

params = {
    "timestamp": int(time.time()),
    "consumer_key": ELE_CONSUMER_KEY,
    "geo":"104.081056,30.660252",
}
url = ELE_HOST + "/restaurants/"
params["sig"] = gen_sig(url, params, ELE_CONSUMER_SECRET)
resp = requests.get(url, params=params).json()
# {'code': 200, 'data': {'cities': [{'abbr': 'SH', 'id': 1, 'name': '上海'},...]}}
print(resp)
print(resp['data'])

In [ ]:
# 查询餐厅信息restaurant-get
# 根据指定的ID查询餐厅信息
params = {
    "timestamp": int(time.time()),
    "consumer_key": ELE_CONSUMER_KEY
}
url = ELE_HOST + "/restaurant/{}/".format(2108634)
params["sig"] = gen_sig(url, params, ELE_CONSUMER_SECRET)
resp = requests.get(url, params=params).json()
# {'code': 200, 'data': {'cities': [{'abbr': 'SH', 'id': 1, 'name': '上海'},...]}}
print(resp)
pprint(resp['data'])

In [ ]:
# 查询餐厅起送价restaurant-deliver-amount
# 根据地理位置坐标查询某一个指定餐厅的起送价
params = {
    "timestamp": int(time.time()),
    "consumer_key": ELE_CONSUMER_KEY,
    "geo": "1200.181855,30.657077"
}
url = ELE_HOST + "/restaurant/{}/deliver_amount/".format(156282495)
params["sig"] = gen_sig(url, params, ELE_CONSUMER_SECRET)

resp = requests.get(url, params=params)
resp_data=resp.json()
pprint(resp_data)
# print(resp_data['data'])

In [ ]:
# 翼支付状态推送
url="http://e-commerce.test.bestpay.net"
channelCode="el_00001"
import requests
import json
import arrow
body={
  "traceLogId":"bp202197824742121",
  "model":{
        "requestId": "chanelCode{}".format(arrow.now().format("YYYYMMDDHHmmss")),
        "channelCode": "el_00001",
        "phoneNo": "15214387213",
        "message": "配送中",
        "actionUrl":"https://bestpay.otosaas.com"+"/waimai/orderDetails/{}".format('bp202197824742121')
}
}

# url='https://e-commerce-pre.bestpay.com.cn'
# url='http://116.228.151.161:19107'
url='https://e-commerce.bestpay.com.cn'
requests.post(url+"/electronic/messageNotice",json=body).json()

In [ ]:
import arrow
date=arrow.now().format("YYYYMMDDHHmmss")
date

In [ ]:
[2017-12-16 09:11:28] 26247-INFO state::ele_status():l-113: {'tp_order_id': 'bp202937344269314', 'extra': '{}', 'new_status': '-1', 'eleme_order_id': '3016734133980301468', 'created_at': '1513386688', 'push_action': '2'}
[2017-12-16 09:11:28] 26247-INFO state::ele_status():l-113: {'tp_order_id': 'bp202937344269314', 'extra': '{"invalid_type": 16, "description": "\\u6682\\u65f6\\u65e0\\u6cd5\\u914d\\u9001:\\u914d\\u9001\\u4eba\\u624b\\u4e0d\\u8db3\\uff0c\\u65e0\\u6cd5\\u63d0\\u4f9b\\u914d\\u9001\\u3002"}', 'eleme_order_id': '3016734133980301468', 'created_at': '1513386688', 'push_action': '2', 'new_status': '-1'}

In [ ]:
start = time.mktime((2017,12,27,10,0,0,0,0,0))*1000
end = time.mktime((2017,12,27,13,0,0,0,0,0))*1000
for item in c.pmdb.order_waimai.find({'createdAt':{'$gte':start,'$lt':end},
                                     'status':}):
    print(item)

In [ ]:
with open('/data/logs/compensation.log') as f:
    compensation_log=f.read()
compensation_log[:1000]

In [ ]:
import tablib
import ast
tab_data=tablib.Dataset(headers=['订单号','手机号'])

import re
rc=re.compile('2017-12-27 18:4.*')
for item in re.findall(rc, compensation_log):
    try:
        order_id = ast.literal_eval(item.split(']')[-1].replace(' ',''))['orderId']
    except Exception as e:
        print(e)
    if c.prdb_mast(0).sismember('bestpay_order_list',order_id):
        phone = rrdb.execute_command("jget", "o:{}".format(order_id),"/userPhone")
        tab_data.append([order_id, phone])


In [ ]:
with open('bestpay-12月27日-订单失败用户统计.xlsx', 'wb') as f:
    f.write(tab_data.xlsx)

In [ ]:
import redis
import urllib3
urllib3.disable_warnings()

rrdb = redis.StrictRedis(host='rrdb.localdomain',port=16379,db=0,charset="UTF-8",decode_responses=True)

for order_id in c.prdb_mast(0).sscan_iter('bestpay_order_list'):
    order = json.loads(rrdb.execute_command("jget", "o:{}".format(order_id),"/"))
    if order['status'] in [9, 4, 11]:
        continue
    item={
            "orderId": order['id'],
            "paymentSerial": order['paymentSerial'],
            "orderType": order['orderType'],
            "appCode": order['appCode'],
            "partnerId": order['partnerId']
            }
    resp=order_status_fetch({'orderId':order['id'],'partnerId':order['partnerId']})
    if resp['statusCode'] == 2:
        message={
            "taskName":"worker.ele.order_status_polling",
            "queue":"ele_tasks",
            "args":item
        }
        c.prdb_mast(0).rpush('session_task_hub',message)
    else:
        c.prdb_paym(1).hmset('p:{}:{}'.format(item['appCode'], item['paymentSerial']), {'status':2,'channel':order['channel']})
        c.prdb_paym(1).rpush('compensation',item)

In [ ]:
import redis
import urllib3
urllib3.disable_warnings()

rrdb = redis.StrictRedis(host='rrdb.localdomain',port=16379,db=0,charset="UTF-8",decode_responses=True)
with open('/data/logs/bestpay_ids.txt') as f:
    text_bestpay = f.read()
order_id_list = text_bestpay.replace(' ','').replace('"','').split('\n')
for order_id in order_id_list:
    order = json.loads(rrdb.execute_command("jget", "o:{}".format(order_id),"/"))
    if order['status'] in [9, 4, 11, 7]:
        continue
    item={
            "orderId": order['id'],
            "paymentSerial": order['paymentSerial'],
            "orderType": order['orderType'],
            "appCode": order['appCode'],
            "partnerId": order['partnerId']
            }
    resp=order_status_fetch({'orderId':order['id'],'partnerId':order['partnerId']})
    if resp['statusCode'] == 2:
        message={
            "taskName":"worker.ele.order_status_polling",
            "queue":"ele_tasks",
            "args":item
        }
        c.prdb_mast(0).rpush('session_task_hub',message)
    else:
        c.prdb_paym(1).hmset('p:{}:{}'.format(item['appCode'], item['paymentSerial']), {'status':2,'channel':'bestpay'})
        c.prdb_paym(1).rpush('compensation',item)
        print(item['paymentSerial'])
#         break

In [ ]:
with open('/data/logs/bestpay.txt') as f:
    bestpay=f.read()

In [ ]:
order_id_map_payment={}
for item in bestpay.replace(' ','').split('\n'):
    it=item.split('"')
    try:
        order_id_map_payment[it[1]]=it[-2]
    except:
        print(it)

for order_id in order_id_map_payment:
    try:
        payment_id = c.pmdb.order_waimai.find_one({'id':order_id},{'paidList':1})['paidList'][0]
    except:
        continue
    if payment_id!=order_id_map_payment[order_id]:
        c.pmdb.order_waimai.update_one({'id':order_id},{'$set':{'paidList':[order_id_map_payment[order_id]]}})
        requests.get('http://dpp.boluome.com/order/sync?orderId={}'.format(order_id))
        print(order_id,payment_id,order_id_map_payment[order_id])